In [1]:
#Step 1: Import Necessary Libraries
import torch
import random
random.seed(42)
from torch import nn
from torch.utils.data import DataLoader
from transformers import BertForMaskedLM, BertTokenizer, AdamW
from datasets import load_dataset,Dataset
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor
print("imported")

imported


In [35]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device")

device

In [36]:
#Step 2: Define Feed-Forward Layer
class FeedForwardLayer(nn.Module):
    def __init__(self, bert_dim, hidden_dim=256):
        super().__init__()
        self.ff = nn.Sequential(
            nn.Linear(bert_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, cls_embedding):
        return self.ff(cls_embedding)
print("ffl class")

ffl class

In [37]:
#Step 3: Load Pre-trained BERT and Tokenizer
bert_model = BertForMaskedLM.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
print("loaded")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).

loaded

In [41]:
#Step 4: Load HotpotQA Dataset 
hotpotqa_corpus = load_dataset("BeIR/hotpotqa", "corpus")
hotpotqa_queries = load_dataset("BeIR/hotpotqa", "queries")
hotpotqa_qrels = load_dataset("BeIR/hotpotqa-qrels")

hotpotqa_corpus_subset = hotpotqa_corpus["corpus"] # no of docs
hotpotqa_queries_subset = hotpotqa_queries["queries"].select(range(3000))  # k queries extracted
hotpotqa_qrels_subset = hotpotqa_qrels["train"].select(range(6000))  # 2k query - positive doc pairs (if doc extracted) #54,000 query doc pairs 

# Check the structure of each dataset to understand how they are organized
print(f"HotpotQA Corpus structure: {hotpotqa_corpus}")
print(f"HotpotQA Queries structure: {hotpotqa_queries}")
print(f"HotpotQA Qrels structure: {hotpotqa_qrels}")


# Initialize the dictionary to store data of query doc pairs
print("hotpotqa_corpus _subset:", len(hotpotqa_corpus_subset))
print("hotpotqa_queries_subset:", len(hotpotqa_queries_subset))
print("hotpotqa_qrels_subset:  ",len(hotpotqa_qrels_subset))

print("loaded")

HotpotQA Corpus structure: DatasetDict({    corpus: Dataset({        features: ['_id', 'title', 'text'],        num_rows: 5233329    })})HotpotQA Queries structure: DatasetDict({    queries: Dataset({        features: ['_id', 'title', 'text'],        num_rows: 97852    })})HotpotQA Qrels structure: DatasetDict({    train: Dataset({        features: ['query-id', 'corpus-id', 'score'],        num_rows: 170000    })    validation: Dataset({        features: ['query-id', 'corpus-id', 'score'],        num_rows: 10894    })    test: Dataset({        features: ['query-id', 'corpus-id', 'score'],        num_rows: 14810    })})hotpotqa_corpus _subset: 5233329hotpotqa_queries_subset: 3000hotpotqa_qrels_subset:   6000loaded

In [43]:
# NITHYA
#dont run this each time you change queries;qrels size***********

# Step 5: Preprocessing for HotpotQA Dataset
query_doc_pairs = {}

# Pre-index queries and corpus into dictionaries for O(1) lookups
corpus_lookup = {doc["_id"]: doc["text"] for doc in hotpotqa_corpus_subset}
print("corpus_lookup done")

corpus_lookup done

In [44]:
# NITHYA
query_lookup = {query["_id"]: query["text"] for query in hotpotqa_queries_subset}
print("query_lookup done")

# Precompute positive pairs into a set for O(1) lookup
positive_pairs = set((qrel["query-id"], str(qrel["corpus-id"])) for qrel in hotpotqa_qrels_subset if qrel["score"] == 1)

# Convert corpus to a list for random sampling
corpus_list = list(corpus_lookup.keys())
print("entering for loop")

for idx, qrel in enumerate(hotpotqa_qrels_subset):
    if idx % 50 == 0:
        print(idx)
    
    query_id = qrel["query-id"]
    doc_id = str(qrel["corpus-id"])  # Convert int64 to str
    
    # Skip non-relevant pairs
    if qrel["score"] != 1:
        continue
    
    # Check if query and document exist
    if query_id not in query_lookup or doc_id not in corpus_lookup:
        continue

    # Retrieve query and document text
    query_text = query_lookup[query_id]
    doc_text = corpus_lookup[doc_id]
    
    # Add positive query-doc pair to the dictionary
    query_doc_pairs[len(query_doc_pairs)] = {
        "query_id": query_id,
        "query_text": query_text,
        "doc_id": doc_id,
        "doc_text": doc_text,
        "label": 1,
    }

    # Negative sampling with checks
    negative_samples = set()
    while len(negative_samples) < 8:
        # Randomly sample a document
        neg_doc_id = random.choice(corpus_list)
        
        # Skip if the document is already a positive pair for the query
        if (query_id, neg_doc_id) in positive_pairs:
            continue
        
        # Skip if the document is already sampled for this query
        if neg_doc_id in negative_samples:
            continue
        
        negative_samples.add(neg_doc_id)

    # Add negative query-doc pairs to the dictionary
    for neg_doc_id in negative_samples:
        query_doc_pairs[len(query_doc_pairs)] = {
            "query_id": query_id,
            "query_text": query_text,
            "doc_id": neg_doc_id,
            "doc_text": corpus_lookup[neg_doc_id],
            "label": 0,
        }

# Output results
print(f"Extracted {len(query_doc_pairs)} query-document pairs.")

query_lookup doneentering for loop0501001502002503003504004505005506006507007508008509009501000105011001150120012501300135014001450150015501600165017001750180018501900195020002050210021502200225023002350240024502500255026002650270027502800285029002950300030503100315032003250330033503400345035003550360036503700375038003850390039504000405041004150420042504300435044004450450045504600465047004750480048504900495050005050510051505200525053005350540054505500555056005650570057505800585059005950Extracted 54000 query-document pairs.

In [45]:
# Step 6 : convert dict to dataset
# Convert query-doc pairs dictionary to a list of dictionaries
query_doc_pairs_list = list(query_doc_pairs.values())

# Create a Hugging Face Dataset from the list
query_doc_dataset = Dataset.from_list(query_doc_pairs_list)
print("loaded")

loaded

In [ ]:
# #Step 5: preprocess HotpotQA Dataset
# query_doc_pairs = {}
# # Iterate through the qrels subset
# for idx, qrel in enumerate(hotpotqa_qrels_subset):
#     if idx%10 == 0:
#         print(idx)
#     query_id = qrel["query-id"]
#     doc_id = str(qrel["corpus-id"]) # convert int64 to str
#     score = qrel["score"]
    
#     if score != 1 :
#         print("a")
#         continue
    
#     # Retrieve query text and doc text if they exist
#     query_match = hotpotqa_queries_subset.filter(lambda x: x["_id"] == query_id)
#     doc_match = hotpotqa_corpus_subset.filter(lambda x: x["_id"] == doc_id)

#     # Check if both query and doc are found
#     if len(query_match) == 0 or len(doc_match) == 0:
#         print("a")
#         continue  # Skip this qrel entry if either is missing

#     query = query_match[0]
#     doc = doc_match[0]

#     query_text = query["text"]
#     doc_text = doc["text"]
   
#     # Add positive query-doc pair to the dictionary
#     query_doc_pairs[len(query_doc_pairs)] = {
#         "query_id": query_id,
#         "query_text": query_text,
#         "doc_id": doc_id,
#         "doc_text": doc_text,
#         "label": 1,
#     }
    
#     # Negative sampling with check in dictionary
#     negative_samples = []
#     while len(negative_samples) < 8:
#         # Randomly sample a document
#         candidate_neg_doc = random.choice(hotpotqa_corpus_subset)

#         # Check if this doc is already a pair for the current query in the dictionary
#         is_already_paired = any(
#             entry["query_id"] == query_id and entry["doc_id"] == candidate_neg_doc["_id"]
#             for entry in query_doc_pairs.values()
#         )

#         # If not already paired, add it as a negative sample
#         if not is_already_paired:
#             negative_samples.append(candidate_neg_doc)
#     print("while ", idx)
#     # Add negative query-doc pairs to the dictionary
#     for neg_doc in negative_samples:
#         query_doc_pairs[len(query_doc_pairs)] = {
#             "query_id": query_id,
#             "query_text": query_text,
#             "doc_id": neg_doc["_id"],
#             "doc_text": neg_doc["text"],
#             "label": 0,
#         }

# # Output
# print(f"Extracted {len(query_doc_pairs)} query-document pairs.")

In [46]:
# Step 7 : Define the preprocess function
def preprocess_function(example):
    # Tokenize query text and document text together
    tokenized = tokenizer(
        text=example['query_text'],  # Query text
        text_pair=example['doc_text'],  # Document text
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    # Return tokenized values along with the label
    return {
        "input_ids": tokenized["input_ids"].squeeze(0),  # Remove extra batch dimension
        "attention_mask": tokenized["attention_mask"].squeeze(0),
        "label": example['label']  # Include label
    }

hotpotqa = query_doc_dataset.map(preprocess_function)
print(hotpotqa)

Map:   0%|          | 0/54000 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs w

Dataset({    features: ['query_id', 'query_text', 'doc_id', 'doc_text', 'label', 'input_ids', 'attention_mask'],    num_rows: 54000})

In [59]:
#Step 8: Define Training Function
def train_ffl(bert_model, train_data, ffl, batch_size=16, lr=1e-4, patience=1, min_delta=1e-3):
    # Ensure the model is on the correct device
    bert_model.to(device)
    ffl.to(device)
    
    bert_model.eval()  # Freeze BERT during FFL training
    ffl.train()
    
    optimizer = AdamW(ffl.parameters(), lr=lr)
    dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)  #3375
    
    
    best_loss = float('inf')  # Initialize best loss
    patience_counter = 0      # Counter for early stopping
    max_epochs = 5
    
    for epoch in range(1,1+ max_epochs):
        total_loss = 0
        b=0
        for batch in dataloader:
            b=b+1
            if(b%135==0):
                print(b)
            optimizer.zero_grad()
            # Ensure input_ids and attention_mask are stacked into one tensor
            input_ids = torch.stack(batch["input_ids"]).to(device).transpose(0, 1)
            attention_mask = torch.stack(batch["attention_mask"]).to(device).transpose(0, 1)

            # Get CLS embeddings from frozen BERT
            with torch.no_grad():
                outputs = bert_model(input_ids, attention_mask=attention_mask, output_hidden_states=True)
                last_hidden_state = outputs.hidden_states[-1]
                cls_embeddings = last_hidden_state[:, 0, :]

            scores = ffl(cls_embeddings)
            labels = batch["label"].float().to(device).unsqueeze(1)
            loss = nn.MSELoss()(scores, labels)
            
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        # Calculate average loss for the epoch
        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch} completed. Avg Loss: {avg_loss}")
        
        # Early stopping logic
        if best_loss - avg_loss > min_delta:  # Significant improvement
            best_loss = avg_loss
            patience_counter = 0  # Reset patience
            torch.save(ffl.state_dict(), "ffl_weights.pth")
            print("Model improved. Saved at ffl_weights.pth")
        else:
            patience_counter += 1  # Increment patience counter

        # Stop training if patience counter exceeds patience
        if patience_counter >= patience:
            print(f"Early stopping after {epoch} epochs. Best Loss: {best_loss}")
            break
            
    print(f"Training stopped after {epoch} epochs. Best Loss: {best_loss}")

In [60]:
#Step 9: Train and Save the FFL
ffl = FeedForwardLayer(bert_dim=768)  # BERT Base has 768 hidden dimensions
train_ffl(bert_model, hotpotqa, ffl)

135270405540675810945108012151350148516201755189020252160229524302565270028352970310532403375Epoch 1 completed. Avg Loss: 0.06482405634962574Model improved. Saved at ffl_weights.pth135270405540675810945108012151350148516201755189020252160229524302565270028352970310532403375Epoch 2 completed. Avg Loss: 0.04999302091711649Model improved. Saved at ffl_weights.pth13527040554067581094510801215135014851620175518902025216022952430256527002970310532403375Epoch 3 completed. Avg Loss: 0.04400678767422559Model improved. Saved at ffl_weights.pth135270405540675810945108012151350148516201755189020252160229524302565270028352970310532403375Epoch 4 completed. Avg Loss: 0.04051082264000757Model improved. Saved at ffl_weights.pth135270405540675810945108012151350148516201755189020252160229524302565270028352970310532403375Epoch 5 completed. Avg Loss: 0.03764189809733243Model improved. Saved at ffl_weights.pthTraining stopped after 5 epochs. Best Loss: 0.03764189809733243

In [2]:
#Step 10: Load HotpotQA Dataset 
scidocs_corpus = load_dataset("BeIR/scidocs", "corpus")
scidocs_queries = load_dataset("BeIR/scidocs", "queries")
scidocs_qrels = load_dataset("BeIR/scidocs-qrels")

scidocs_corpus_subset = scidocs_corpus["corpus"] # no of docs
scidocs_queries_subset = scidocs_queries["queries"]
scidocs_qrels_subset = scidocs_qrels["test"]

# Initialize the dictionary to store data of query doc pairs
print("scidocs_corpus _subset:", len(scidocs_corpus_subset))
print("scidocs_queries_subset:", len(scidocs_queries_subset))
print("scidocs_qrels_subset:  ",len(scidocs_qrels_subset))

print("loaded")

0000.parquet:   0%|          | 0.00/19.0M [00:00<?, ?B/s]

Generating corpus split:   0%|          | 0/25657 [00:00<?, ? examples/s]

queries/queries/0000.parquet:   0%|          | 0.00/93.3k [00:00<?, ?B/s]

Generating queries split:   0%|          | 0/1000 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/14.0k [00:00<?, ?B/s]

test.tsv:   0%|          | 0.00/2.54M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/29928 [00:00<?, ? examples/s]

scidocs_corpus _subset: 25657
scidocs_queries_subset: 1000
scidocs_qrels_subset:   29928
loaded


In [3]:
# Create dictionaries for fast access to queries and docs
queries_dict = {query["_id"]: query for query in scidocs_queries_subset}
docs_dict = {doc["_id"]: doc for doc in scidocs_corpus_subset}

# Initialize the dictionary to store the query-doc pairs
scidocs_query_doc_pairs = {}

# Iterate through the qrels subset
for idx, qrel in enumerate(scidocs_qrels_subset):
    query_id = qrel["query-id"]
    doc_id = qrel["corpus-id"]
    score = qrel["score"]

    # Retrieve query and doc from the dictionaries
    query = queries_dict.get(query_id)
    doc = docs_dict.get(doc_id)

    # Check if both query and doc are found
    if query is None or doc is None:
        continue  # Skip this qrel entry if either is missing

    # Retrieve texts
    query_text = query["text"]
    doc_text = doc["text"]

    # Add the query-doc pair to the dictionary
    scidocs_query_doc_pairs[len(scidocs_query_doc_pairs)] = {
        "query_id": query_id,
        "query_text": query_text,
        "doc_id": doc_id,
        "doc_text": doc_text,
        "label": score,
    }

# Output the number of query-doc pairs extracted
print(f"Extracted {len(scidocs_query_doc_pairs)} scidocs query-document pairs.")

Extracted 29928 scidocs query-document pairs.


In [ ]:
# #Step 11: preprocess scidocs Dataset
# scidocs_query_doc_pairs = {}
# # Iterate through the qrels subset
# for idx, qrel in enumerate(scidocs_qrels_subset):
#     query_id = qrel["query-id"]
#     doc_id = qrel["corpus-id"]
#     score = qrel["score"]
    
#     # Retrieve query text and doc text if they exist
#     query_match = scidocs_queries_subset.filter(lambda x: x["_id"] == query_id)
#     doc_match = scidocs_corpus_subset.filter(lambda x: x["_id"] == doc_id)

#     # Check if both query and doc are found
#     if len(query_match) == 0 or len(doc_match) == 0:
#         print("a")
#         continue  # Skip this qrel entry if either is missing

#     query = query_match[0]
#     doc = doc_match[0]

#     query_text = query["text"]
#     doc_text = doc["text"]
   
#     # Add positive query-doc pair to the dictionary
#     scidocs_query_doc_pairs[len(scidocs_query_doc_pairs)] = {
#         "query_id": query_id,
#         "query_text": query_text,
#         "doc_id": doc_id,
#         "doc_text": doc_text,
#         "label": score,
#     }
#     # 5 Negative samples for each positive sample are already included here 
# print(f"Extracted {len(scidocs_query_doc_pairs)} scidocs query-document pairs.")

In [63]:
# Step 12 : convert dict to dataset
# Convert query-doc pairs dictionary to a list of dictionaries
scidocs_query_doc_pairs_list = list(scidocs_query_doc_pairs.values())

# Create a Hugging Face Dataset from the list
scidocs_query_doc_dataset = Dataset.from_list(scidocs_query_doc_pairs_list)
print("loaded")

loaded

In [5]:
def preprocess_scidocs(example):
    # Tokenize query text and document text together
    tokenized = tokenizer(
        text=example['query_text'],  # Query text
        text_pair=example['doc_text'],  # Document text
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors="pt",
    )
    # Return tokenized values along with the label
    return {
        "input_ids": tokenized["input_ids"],
        "attention_mask": tokenized["attention_mask"],
        "label": example['label']  # Include label
    }
    
scidocs = scidocs_query_doc_dataset.map(preprocess_scidocs)
print(scidocs)

Dataset({
    features: ['query_id', 'query_text', 'doc_id', 'doc_text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 29928
}))


In [65]:
#Step 8: Define Masked Language Modeling Step
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

def masked_language_modeling_step(model, batch):
    device = model.device
    inputs = torch.tensor(batch["input_ids"]).to(device)
    attention_mask = torch.tensor(batch["attention_mask"]).to(device)
    # Prepare features with labels for MLM (labels are the same as inputs)
    features = [{"input_ids": inputs, "attention_mask": attention_mask}]
    collated_batch = data_collator(features)
    # Extract input_ids and labels from the collated batch
    inputs = collated_batch["input_ids"].squeeze(0).to(device)
    labels = collated_batch["labels"].squeeze(0).to(device)
    outputs = model(input_ids=inputs, attention_mask=attention_mask, labels=labels)
    loss = outputs.loss
    return loss
print("MLM")

MLM

In [66]:
# Step 9: Inference with Test-Time Training
original_state = bert_model.state_dict()

def test_time_inference(bert_model, ffl, scidocs, data_collator,K , masked):
    bert_model.to(device)
    ffl.to(device)
    
    bert_model.train()  # Enable temporary updates
    ffl.eval()          # No updates to the FFL

    optimizer = AdamW(bert_model.parameters(), lr=1e-5)
    
    scores = {}
    count = 0
    for example in scidocs:
        count = count + 1
        
        if masked == 1:
            #Masked reconstruction
            loss = masked_language_modeling_step(bert_model, example)
    
            # Backward pass and optimization
            optimizer.zero_grad()  
            loss.backward()        
            optimizer.step()
        
        # Inference
        input_ids = torch.tensor(example["input_ids"]).to(device)  # Ensure tensors
        attention_mask = torch.tensor(example["attention_mask"]).to(device)  # Ensure tensors
        bert_model.eval()
        with torch.no_grad():
            outputs = bert_model(input_ids, attention_mask=attention_mask, output_hidden_states=True)
            last_hidden_state = outputs.hidden_states[-1]
            cls_embeddings = last_hidden_state[:, 0, :]
            score = ffl(cls_embeddings)

            # Add positive query-doc pair and score to the dictionary
            scores[len(scores)] = {
                "query_id": example["query_id"],
                "doc_id": example["doc_id"],
                "label": example["label"],
                "score": score.item(),
            }
        
        bert_model.train()
        if masked == 1 : 
            #  Restore original weights
            if count%K == 0:
                bert_model.load_state_dict(original_state)
    
    return scores

In [69]:
# Group scores by query_id
def metrics(scores,K,masked):
    grouped_scores = defaultdict(list)
    for entry in scores.values():
        grouped_scores[entry["query_id"]].append(entry)
        
    # Sort scores for each query_id and retrieve corresponding labels
    results_by_query = {}
    for query_id, entries in grouped_scores.items():
        
        # Sort entries by score in descending order
        sorted_entries = sorted(entries, key=lambda x: x["score"], reverse=True)
        results_by_query[query_id] = [entry["label"] for entry in sorted_entries]
    
    MRR = []
    P10 = []
    R10 = []

    for query_id, data in results_by_query.items():
        
        m = 1.0/(data.index(1) +1)
        MRR.append(m)
    
        rgc = data[:10].count(1)
        tgc = data.count(1)
        
        p = rgc/10
        P10.append(p)
        r = rgc/tgc
        R10.append(r)
    
    l = len(results_by_query)
    avg_mrr = sum(MRR) / l
    avg_p = sum(P10) / l
    avg_r = sum(R10) / l
    if masked == 1:
        print(f"No of queries tested is {l} and granularity K is {K} ")
    elif masked == 0:
        print(f"No of queries tested is {l} and with normal interference ")
    print(f"MRR : {avg_mrr} , P@10 : {avg_p} , R@10 : {avg_r}")

In [6]:
# Step 10: Run the Inference

ffl.load_state_dict(torch.load("ffl_weights.pth", weights_only=True))  # Load trained FFL weights with weights_only=True

#without mlm reconstruction
scores = test_time_inference(bert_model, ffl, scidocs, data_collator,1,0)
metrics(scores,1 , 0)

K = [1,8,16,64]  #granularity
for k in K :
    scores = test_time_inference(bert_model, ffl, scidocs, data_collator,k,1)
    metrics(scores,k,1)

No of queries tested is 1000 and with normal interference
MRR : 0.253687438715857 , P@10 : 0.13432835820895522 , R@10 : 0.2776119402985075
No of queries tested is 1000 and granularity K is 1 
MRR : 0.26664535713043175 , P@10 : 0.14328358208955216 , R@10 : 0.2970149253731342
No of queries tested is 1000 and granularity K is 8 
MRR : 0.2424190298531826 , P@10 : 0.13432835820895514 , R@10 : 0.2791044776119402
No of queries tested is 1000 and granularity K is 16 
MRR : 0.21196034934829347 , P@10 : 0.12537313432835817 , R@10 : 0.2611940298507462
No of queries tested is 1000 and granularity K is 64
MRR : 0.22158670209129977 , P@10 : 0.126865671641791 , R@10 : 0.2634328358208954
